In [65]:
import yfinance as yf
import numpy as np, pandas as pd
import math
from datetime import datetime,timedelta
import matplotlib.pyplot as plt
from scipy import stats
from scipy.optimize import minimize

In [66]:
i=complex(0,1)

In [67]:
def d(rho, sigma, x):
    a=(rho*sigma*i*x)**2
    b=(sigma**2)*(i*x +x**2)
    return (a+b)**0.5

In [68]:
def g(kappa, rho, sigma, x):
    num=kappa - sigma*rho*i*x +d(rho, sigma, x)
    den=kappa - sigma*rho*i*x -d(rho, sigma, x)
    return num/den

In [69]:
def fHeston(x,r,t,S,rho,kappa, sigma,theta,v):
    p1 = np.exp(i*x*r*t)*S**(i*x)
    p2=((1-g(kappa, rho, sigma, x)*np.exp(t*d(rho, sigma, x)))/(1-g(kappa, rho, sigma, x)))**(-2*kappa*theta/(sigma**2))
    p3=(t*kappa*theta/(sigma**2))*(kappa - sigma*rho*i*x -d(rho, sigma,x))
    p4=(v/(sigma**2))*(kappa - sigma*rho*i*x -d(rho, sigma, x))*((1-np.exp(-d(rho, sigma, x)*t))/(1-g(kappa, rho, sigma, x)*np.exp(t*-d(rho, sigma, x))))
    return p1*p2*np.exp(p3+p4)

In [70]:
def intHeston(x,r,t,K,kappa,S,rho, sigma, theta,v):
    P, iterations, maxnum = 0, 1000, 100
    ds=maxnum/iterations
    for j in range(1,iterations):
        s1 = ds*(2*j+1)/2
        s2 = s1-i
        num1 = np.exp(r*t)*fHeston(s2,r,t,S,rho,kappa, sigma,theta,v)
        num2 = K*fHeston(s1,r,t,S,rho,kappa, sigma,theta,v)
        den=np.exp(np.log(K) *i*s1)*i*s1
        P += ds*(num1-num2)/den
    P/=np.pi
    P0 = 0.5*(S - K*np.exp(-r*t))
    return np.real(P+P0)

In [71]:
def get_t(date):
    date1= datetime.strptime(date, '%Y-%m-%d')
    date2 = datetime.now()
    return (date1-date2).days/365.25

In [72]:
def getS(symbol):
    stock = yf.Ticker(symbol)
    return stock.history(period="1d")['Close'].iloc[-1]

In [73]:
def getr():
    symbol="^TNX"
    now=datetime.now()
    ago10=now.replace(year=now.year-10)
    data = yf.download(symbol,ago10,now)
    return (data['Close'].iloc[-1])/100

In [74]:
def getx(symbol):
    return np.log(getS(symbol))

In [75]:
def optiondata(symbol,date):
    option = yf.Ticker(symbol)
    option = option.option_chain(date)
    return option.calls, option.puts

In [76]:
def getvol(symbol):
    data = yf.download(symbol, start=datetime.now().replace(year=datetime.now().year-10), end=datetime.now())
    data['v1']=data['Adj Close'].pct_change()
    data['v2']=data['v1'].pct_change()
    return (data['v1'].std())*252**0.5

In [78]:
def objective(vars, symbol):
    k,r,sigma,theta=vars
    mu1 = 1 + r
    mu2 = (r + 1)**2 + theta
    mu4 = (1 / (k * (k - 2))) * (
        k**2 * r**4 + 4 * k**2 * r**3 + 6 * k**2 * r**2 * theta - 2 * k * r**4 + 6 * k**2 * r**2 + 12 * k**2 * r * theta
        + 3 * k**2 * theta**2 - 8 * k * r**3 - 12 * k * r * theta + 4 * k**2 * r + 6 * k**2 * theta - 12 * k * r**2
        - 24 * k * r * theta - 6 * k * theta**2 - 3 * sigma**2 * theta + k**2 - 8 * k * r - 12 * k * theta - 2 * k
    )
    mu5 = (1 / (k * (k - 2))) * (
        k**2 * r**5 + 5 * k**2 * r**4 + 10 * k**2 * r**3 * theta - 2 * k * r**5 + 10 * k**2 * r**3 + 30 * k**2 * r**2 * theta
        + 15 * k**2 * r * theta**2 - 10 * k * r**4 - 20 * k * r**3 * theta + 10 * k**2 * r**2 + 30 * k**2 * r * theta + 15 * k**2 * theta**2
        - 20 * k * r**3 - 60 * k * r**2 * theta - 30 * k * r * theta**2 - 15 * sigma**2 * theta + 5 * k**2 * r + 10 * k**2 * theta
        - 20 * k * r**2 - 60 * k * r * theta - 30 * k * theta**2 - 15 * sigma**2 * theta + k**2 - 10 * k * r - 20 * k * theta - 2 * k
    )
    mus=[]
    data = yf.download(symbol, start=datetime.now().replace(year=datetime.now().year-10), end=datetime.now())
    data['change']=data['Adj Close'].pct_change()
    for i in range(5):
        mus.append(np.mean((data['change']+1)**(i+1)))
    return (mu1-mus[0])**2 + (mu2-mus[1])**2+(mu4-mus[3])**2+(mu5-mus[4])**2

In [79]:
def minimize_fun(symbol):
    constraints=({'type':'ineq','fun':lambda vars:vars[2]**2},
                {'type':'ineq','fun':lambda vars:2*vars[0]*vars[3]-vars[2]**2})   
    initial_guess=[1,1,1,1]
    result=minimize(objective,initial_guess,args=(symbol),method='SLSQP',options={'disp':False}, constraints=constraints)
    return result.x

In [112]:
symbol='MSFT'
results=minimize_fun(symbol)
option=yf.Ticker(symbol)
date=option.options[1]
calls,puts=optiondata(symbol,date)
col_drop=['lastTradeDate', 'lastPrice', 'volume', 'openInterest', 'contractSize', 'currency']
calls.drop(col_drop, axis=1, inplace=True)
r=getr() #risk free rate
x=getx(symbol) #log of stock price
sigma=getvol(symbol) #volatility
v=results[2] #volatility of volatility
S=getS(symbol) #stock price
t=get_t(date) #time to expiry
kappa = results[0] #mean reverting rate
theta = results[3] #long term mean volatility
rho = 0.5 #correlation between stock price and volatility
calls['HestonValuation']=calls.apply(lambda row:intHeston(x,r,t,row['strike'],kappa,S,rho, sigma, theta,v),axis=1)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [113]:
results

array([1.77666760e+00, 1.14954284e-03, 3.21130042e-02, 2.90154683e-04])

In [114]:
calls

,contractSymbol,strike,bid,ask,change,percentChange,impliedVolatility,inTheMoney,HestonValuation
0,MSFT240628C00220000,220.0,224.50,228.45,0.000000,0.000000,1.658205,True,220.476640
1,MSFT240628C00230000,230.0,194.15,196.55,0.000000,0.000000,0.000010,True,211.161695
2,MSFT240628C00240000,240.0,205.20,208.50,4.549988,2.254143,1.735353,True,201.991433
3,MSFT240628C00250000,250.0,194.55,198.50,25.539993,14.848833,1.429690,True,192.083799
4,MSFT240628C00260000,260.0,184.50,188.50,0.000000,0.000000,1.318363,True,182.695299
...,...,...,...,...,...,...,...,...,...
56,MSFT240628C00505000,505.0,0.03,0.07,-0.040000,-44.444447,0.312507,False,-1.679095
57,MSFT240628C00510000,510.0,0.03,0.07,-0.010000,-12.499998,0.333991,False,-1.533961
58,MSFT240628C00515000,515.0,0.01,0.06,0.000000,0.000000,0.349616,False,-1.675924
59,MSFT240628C00520000,520.0,0.01,0.06,0.000000,0.000000,0.370123,False,-2.032690
